# EDA for March Learning Mania

### Goal: 
We want to create a model which can provide predictions in the format of YEAR_TEAM1ID_TEAM2ID_WINCHANCE

where the winchance is a value from 0->1 representing the probability that the team with the higher team ID will win. 

"For example, "2025_1101_1102" indicates a hypothetical matchup between team 1101 and 1102 in the year 2025. You must predict the probability that the team with the lower TeamId beats the team with the higher TeamId"

Output should look like this:
```
ID,Pred
2025_1101_1102,0.5
2025_1101_1103,0.5
2025_1101_1104,0.5
...
```

### Data files:
They give you a suffocating amount of data, including: 
- Basic Team & Season Data
- Detailed Game Statistics
- Geographical Data
- Ranking & Rating Systems
- Tournament Bracket Structure
- Supplementary Data
- Data for Submissions
- Miscellaneous

GPT summarized them in the blob below. The key point is that you should probably be able to build a basic model with the basic team and season data. 

Based on historical matches between teams, how will they perform in the future?

The first steps will to get the data into basic form

#### **Basic Team & Season Data**
- **MTeams.csv / WTeams.csv** – Lists all NCAA teams with unique Team IDs and historical Division-I participation details.
- **MSeasons.csv / WSeasons.csv** – Details past seasons, including season start dates and region assignments.
- **MNCAATourneySeeds.csv / WNCAATourneySeeds.csv** – Provides historical NCAA tournament seedings for teams since 1985 (men) and 1998 (women).
- **MRegularSeasonCompactResults.csv / WRegularSeasonCompactResults.csv** – Contains game results (winner, loser, score, location) for regular season games since 1985 (men) and 1998 (women).
- **MNCAATourneyCompactResults.csv / WNCAATourneyCompactResults.csv** – Similar to above but specific to NCAA tournament games.

#### **Detailed Game Statistics**
- **MRegularSeasonDetailedResults.csv / WRegularSeasonDetailedResults.csv** – Contains extended game stats like field goals, rebounds, assists, etc., since 2003 (men) and 2010 (women).
- **MNCAATourneyDetailedResults.csv / WNCAATourneyDetailedResults.csv** – Similar to above but specific to NCAA tournament games.

#### **Geographical Data**
- **Cities.csv** – Lists cities where games were played, including city IDs, names, and state abbreviations.
- **MGameCities.csv / WGameCities.csv** – Maps each game to its city, starting from 2010.

#### **Ranking & Rating Systems**
- **MMasseyOrdinals.csv** – Weekly rankings of men's teams across various rating systems since 2003.

#### **Tournament Bracket Structure**
- **MNCAATourneySlots.csv / WNCAATourneySlots.csv** – Defines how teams progress through the tournament based on seed matchups.
- **MNCAATourneySeedRoundSlots.csv** – Maps tournament seeds to their expected bracket slots and game rounds (men only).

#### **Supplementary Data**
- **MTeamCoaches.csv** – Lists head coaches for teams per season, including mid-season changes.
- **Conferences.csv** – Contains NCAA conference names and abbreviations.
- **MTeamConferences.csv / WTeamConferences.csv** – Tracks which teams belonged to which conferences each season.
- **MConferenceTourneyGames.csv / WConferenceTourneyGames.csv** – Identifies games from conference tournaments before the NCAA tournament.
- **MSecondaryTourneyTeams.csv / WSecondaryTourneyTeams.csv** – Lists teams that participated in secondary postseason tournaments (e.g., NIT).
- **MSecondaryTourneyCompactResults.csv / WSecondaryTourneyCompactResults.csv** – Contains results for games in secondary tournaments.

#### **Data for Submissions**
- **SampleSubmissionStage1.csv / SampleSubmissionStage2.csv** – Example submission files showing expected format.
- **SeedBenchmarkStage1.csv** – Baseline model predictions based on seed matchups.

#### **Miscellaneous**
- **MTeamSpellings.csv / WTeamSpellings.csv** – Helps map external team name variations to standardized Team IDs.


## Organize data

In [21]:
import pandas as pd
import yaml

with open('config.yaml', 'r') as file:
    config_file = yaml.safe_load(file)
data_dir = config_file.get("data_dir")

def peek(file_name):
    df = pd.read_csv(f"{data_dir}/{file_name}.csv").head()
    return(df)

In [22]:
# Straight forward, mostly a lookup table. 
peek("/Kaggle/MTeams")

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2025
1,1102,Air Force,1985,2025
2,1103,Akron,1985,2025
3,1104,Alabama,1985,2025
4,1105,Alabama A&M,2000,2025


In [23]:
# A bit confusing, will take a bit of work to get the seeds lined up, but maybed they've already done it for us
peek("/Kaggle/MNCAATourneySeeds")

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [24]:
## So this has a lot of potential. 
# You could get a better sense of a probability of a win based on team-by-team point differentials. 
# The model should also see how "good" the team is that season, based on this differential.
peek("/Kaggle/MRegularSeasonCompactResults")

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [ ]:
# Exact same as above, but for the tournament.
# Still not sure how to use the daynumber to understand which game it is in the tournament.
# I'd like to include some "heating up" factor, becuase it seems teams go on runs, but maybe that's just a function of media
# aspirational if anything
peek("/Kaggle/MNCAATourneyCompactResults")

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [25]:
# More data than I know what to do with...
peek("/Kaggle/MRegularSeasonDetailedResults")

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


### Actual EDA
On the topic of actual EDA and looking at the shape of the data, this person has already done a lot of it:
[BBall EDA Tutorial](https://www.kaggle.com/code/clehmann10/bball-eda-tutorial)

## Model Development!

I liked this approach to splitting up the submission to get the exact teams you'll need: [notebook](https://www.kaggle.com/code/paultimothymooney/simple-starter-notebook-for-march-mania-2025)

However, the model is very basic (just a function of difference_in_seed*.03+.5 to affect probability.), this [notebook is a bit better and shows building a random forest model](https://www.kaggle.com/code/jocelyndumlao/march-ml-mania-2025-brier-score-prediction)

In [67]:
import numpy as np

# Load the seeds, make them into one dataframe, and fill in the missing values with 0.05 (not sure why not 0)
# w_seed = pd.read_csv(f'{data_dir}/Kaggle/WNCAATourneySeeds.csv')
# m_seed = pd.read_csv(f'{data_dir}/Kaggle/MNCAATourneySeeds.csv')

# Import the submission file
submission_df = pd.read_csv(f'{data_dir}/Kaggle/SampleSubmissionStage2.csv')
submission_df.head()

,ID,Pred
0,2025_1101_1102,0.5
1,2025_1101_1103,0.5
2,2025_1101_1104,0.5
3,2025_1101_1105,0.5
4,2025_1101_1106,0.5


In [68]:
def extract_game_info(id_str):
    # Extract year and team_ids
    parts = id_str.split('_')
    year = int(parts[0])
    teamID1 = int(parts[1])
    teamID2 = int(parts[2])
    return year, teamID1, teamID2

def extract_seed_value(seed_str):
    # Extract seed value
    try:
        return int(seed_str[1:])
    # Set seed to 16 for unselected teams and errors
    except ValueError:
        return 16

In [69]:
# Reformat the data
submission_df[['Season', 'TeamID1', 'TeamID2']] = submission_df['ID'].apply(extract_game_info).tolist()

In [ ]:
# Now we have the data in a more usable format, we can merge it with elo to get the expected outcome
submission_df

,ID,Pred,Season,TeamID1,TeamID2
0,2025_1101_1102,0.5,2025,1101,1102
1,2025_1101_1103,0.5,2025,1101,1103
2,2025_1101_1104,0.5,2025,1101,1104
3,2025_1101_1105,0.5,2025,1101,1105
4,2025_1101_1106,0.5,2025,1101,1106
...,...,...,...,...,...
131402,2025_3477_3479,0.5,2025,3477,3479
131403,2025_3477_3480,0.5,2025,3477,3480
131404,2025_3478_3479,0.5,2025,3478,3479
131405,2025_3478_3480,0.5,2025,3478,3480


## Get Nate Data
https://www.natesilver.net/p/sbcb-college-basketball-ratings-men

In [103]:
nate = pd.read_csv(f'{data_dir}/Nate/data-nO2M0.csv', index_col=0)
nate.head()

,Team,Conf.,Current Elo,Last,Season Min.,Season Max.,Home Court*
1,Duke,ACC,2089.303,+14 🟢@@13.77758789,1842.133,2089.303,83.04591
2,Houston,Big 12,2074.552,+7 🟢@@6.890441895,1863.668,2074.552,84.47469
3,Auburn,SEC,2049.707,-18 🟠@@-17.93383789,1863.583,2105.443,67.58400
4,Florida,SEC,2040.319,+7 🟢@@6.597717285,1799.165,2040.319,62.82391
5,Alabama,SEC,2031.764,+18 🟢@@17.93389893,1903.559,2086.004,100.50200


I'll need to match the nate table to the kaggle names, and I'll use fuzzywuzzy for that
Right now he only has mens teams, so for womens I'll need to derive ELO myself using a simple historic win-loss calculator I think

In [104]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [105]:
MTeamSpellings = pd.read_csv(f"{data_dir}/Kaggle/MTeamSpellings.csv")
choices = list(MTeamSpellings["TeamNameSpelling"])
# Example of how this will work:
process.extractOne(nate['Team'][1], choices)

('duke', 100)

In [106]:
nate['kaggle_name'] = nate['Team'].apply(lambda x: process.extractOne(x, choices)[0])
nate['fuzz_score'] = nate['Team'].apply(lambda x: process.extractOne(x, choices)[1])
# check for erroneous matches by going worst to best
display(nate[['kaggle_name','Team','fuzz_score']].sort_values(by='fuzz_score', ascending=True))
# In my case, the only error was Miami University (OH), which got matches with 'uni'
nate.loc[nate['kaggle_name'] == 'uni', 'kaggle_name'] = 'miami (oh)'

,kaggle_name,Team,fuzz_score
300,texas rio grande valley,UT Rio Grande Valley,88
211,umass,UMass (Amherst),90
167,uni,Miami University (OH),90
215,queens (nc),Queens,90
364,mississippi valley state,Mississippi Valley St.,93
...,...,...,...
118,middle tennessee,Middle Tennessee,100
117,tcu,TCU,100
116,uab,UAB,100
114,furman,Furman,100


In [107]:
nate

,Team,Conf.,Current Elo,Last,Season Min.,Season Max.,Home Court*,kaggle_name,fuzz_score
1,Duke,ACC,2089.3030,+14 🟢@@13.77758789,1842.1330,2089.303,83.04591,duke,100
2,Houston,Big 12,2074.5520,+7 🟢@@6.890441895,1863.6680,2074.552,84.47469,houston,100
3,Auburn,SEC,2049.7070,-18 🟠@@-17.93383789,1863.5830,2105.443,67.58400,auburn,100
4,Florida,SEC,2040.3190,+7 🟢@@6.597717285,1799.1650,2040.319,62.82391,florida,100
5,Alabama,SEC,2031.7640,+18 🟢@@17.93389893,1903.5590,2086.004,100.50200,alabama,100
...,...,...,...,...,...,...,...,...,...
360,Chicago St.,NSEC,1082.9020,-3 🟠@@-2.836730957,1082.9020,1293.097,23.51743,chicago st,100
361,Arkansas-Pine Bluff,SWAC,1055.6120,-7 🟠@@-6.626464844,995.7072,1209.382,29.78964,arkansas-pine bluff,100
362,New Orleans,Southland,1049.2200,-4 🟠@@-4.07635498,1049.2200,1218.143,47.22064,new orleans,100
363,Prairie View A&M,SWAC,1027.2520,-8 🟠@@-8.157409668,1027.2520,1215.821,40.47441,prairie view a&m,100


In [108]:
# Merge the two to get the team_id
merged = pd.merge(nate, MTeamSpellings, left_on='kaggle_name', right_on='TeamNameSpelling', how='left')
# Double check the df looks good
merged

,Team,Conf.,Current Elo,Last,Season Min.,Season Max.,Home Court*,kaggle_name,fuzz_score,TeamNameSpelling,TeamID
0,Duke,ACC,2089.3030,+14 🟢@@13.77758789,1842.1330,2089.303,83.04591,duke,100,duke,1181
1,Houston,Big 12,2074.5520,+7 🟢@@6.890441895,1863.6680,2074.552,84.47469,houston,100,houston,1222
2,Auburn,SEC,2049.7070,-18 🟠@@-17.93383789,1863.5830,2105.443,67.58400,auburn,100,auburn,1120
3,Florida,SEC,2040.3190,+7 🟢@@6.597717285,1799.1650,2040.319,62.82391,florida,100,florida,1196
4,Alabama,SEC,2031.7640,+18 🟢@@17.93389893,1903.5590,2086.004,100.50200,alabama,100,alabama,1104
...,...,...,...,...,...,...,...,...,...,...,...
362,Arkansas-Pine Bluff,SWAC,1055.6120,-7 🟠@@-6.626464844,995.7072,1209.382,29.78964,arkansas-pine bluff,100,arkansas-pine bluff,1115
363,Arkansas-Pine Bluff,SWAC,1055.6120,-7 🟠@@-6.626464844,995.7072,1209.382,29.78964,arkansas-pine bluff,100,arkansas-pine bluff,1115
364,New Orleans,Southland,1049.2200,-4 🟠@@-4.07635498,1049.2200,1218.143,47.22064,new orleans,100,new orleans,1309
365,Prairie View A&M,SWAC,1027.2520,-8 🟠@@-8.157409668,1027.2520,1215.821,40.47441,prairie view a&m,100,prairie view a&m,1341


## sample submission
Now let's do a simple submission based on just nates elo scores

In [112]:
# We need to join Current Elo with the submissions table, and calculate the expected outcome
submission_df['TeamID1']
# merged.set_index('TeamID')['Current Elo']

0         1101
1         1101
2         1101
3         1101
4         1101
          ... 
131402    3477
131403    3477
131404    3478
131405    3478
131406    3479
Name: TeamID1, Length: 131407, dtype: int64

In [129]:
# Create a dictionary for quick lookup of ELO ratings by TeamID
elo_dict = merged.set_index('TeamID')['Current Elo'].to_dict()

# Map the ELO ratings to the TeamID1 column in the submission_df
submission_df['TeamID1_Elo'] = submission_df['TeamID1'].map(elo_dict)
submission_df['TeamID2_Elo'] = submission_df['TeamID2'].map(elo_dict)

# Fill missing values with 1500 - these are mostly womens teams that are not in the dataset yet
submission_df['TeamID1_Elo'].fillna(1500, inplace=True)
submission_df['TeamID2_Elo'].fillna(1500, inplace=True)

# Check the result
submission_df


/var/folders/xb/mmrwx40x1xl_0gj4ldtrzqq40000gn/T/ipykernel_44963/4174457450.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  submission_df['TeamID1_Elo'].fillna(1500, inplace=True)
/var/folders/xb/mmrwx40x1xl_0gj4ldtrzqq40000gn/T/ipykernel_44963/4174457450.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting val

,ID,Pred,Season,TeamID1,TeamID2,TeamID1_Elo,TeamID2_Elo
0,2025_1101_1102,0.5,2025,1101,1102,1482.911,1267.390
1,2025_1101_1103,0.5,2025,1101,1103,1482.911,1635.409
2,2025_1101_1104,0.5,2025,1101,1104,1482.911,2031.764
3,2025_1101_1105,0.5,2025,1101,1105,1482.911,1109.851
4,2025_1101_1106,0.5,2025,1101,1106,1482.911,1321.198
...,...,...,...,...,...,...,...
131402,2025_3477_3479,0.5,2025,3477,3479,1500.000,1500.000
131403,2025_3477_3480,0.5,2025,3477,3480,1500.000,1500.000
131404,2025_3478_3479,0.5,2025,3478,3479,1500.000,1500.000
131405,2025_3478_3480,0.5,2025,3478,3480,1500.000,1500.000


In [147]:
# This should be replaced with goto conversion thing
def calc_elo_win(A, B):
    awin = 1 / (1 + 10**( (B - A) / 400))
    return(awin)
submission_df['Team1_win_prob'] = submission_df.apply(lambda x: calc_elo_win(x['TeamID1_Elo'], x['TeamID2_Elo']), axis=1)

In [148]:
finalsub = submission_df[['ID', 'Team1_win_prob']]

In [149]:
finalsub

,ID,Team1_win_prob
0,2025_1101_1102,0.775675
1,2025_1101_1103,0.293624
2,2025_1101_1104,0.040720
3,2025_1101_1105,0.895435
4,2025_1101_1106,0.717257
...,...,...
131402,2025_3477_3479,0.500000
131403,2025_3477_3480,0.500000
131404,2025_3478_3479,0.500000
131405,2025_3478_3480,0.500000
